### Initial Comment

This script aims to piece together all the data obtained from the webscrapping tools to allow the correct production of the dashboard based on the defined template.

To do so this script will follow the following steps:
 - Import all the data from the three supermarkets and will store these data in different dataframes
 - Remove the first two rows (headers) and add the correct column names
 - Fill the missing data for each supermarket based on the supermarket and the category that they belong to
 - Replace the words that were changed regarding 'Bovino' to 'Vaca' as per the initial criteria
 - Fill the product category information based on the first availble value for each row
 - Left join each data frame with the original set of categories so that if there are missing products, all of them are placed on the DataFrame
 - Concatenate all dataframes into one in which each product is repeated three times and has 150 rows

In [1]:
import pandas as pd

In [2]:
# Import all the data
continente = pd.read_excel('Continente_ProductData.xlsx')
auchan = pd.read_excel('Auchan_Products.xlsx')
minipreco = pd.read_excel('MiniPreco_Products.xlsx')

In [3]:
row_names = ['Supermarket','Category Object','Product Object',
             'Average Per Product','Min Price per Product','Max Price per Product','Count of Products']

In [4]:
# Rename the columns
continente.columns = row_names
auchan.columns = row_names
minipreco.columns = row_names

# Remove the first two rows of each df
continente = continente.iloc[2:,:]
auchan = auchan.iloc[2:,:]
minipreco = minipreco.iloc[2:,:]

In [5]:
# Add the supermarket info to the 'Supermarket' column
continente["Supermarket"] = "Continente"
auchan["Supermarket"] = "Auchan"
minipreco["Supermarket"] ="Mini Preço"

In [6]:
# Replace the string 'Bovino' for 'Vaca' as per the initial criteria in Auchan and Mini Preço
auchan['Product Object'] =  auchan['Product Object'].replace("Bovino", "Vaca")
minipreco['Product Object'] = minipreco['Product Object'].replace("Bovino", "Vaca")


In [7]:
## Define a function to fill all the NaN on the Category Object columm

def remove_NaN(dataframe):
    copy = dataframe.copy()
    indexes_wData = copy[~copy["Category Object"].isna()].index
    
    for index, row in copy.iterrows():
        if index in indexes_wData:
            last_categoryName = copy.loc[index,"Category Object"] # Store the last known value for the category name
        else:
            dataframe.loc[index, "Category Object"] = last_categoryName # Passes the last known category name to the original df
    
    return dataframe

In [8]:
# Clean each data frame from the NaN
continente = remove_NaN(continente)
auchan = remove_NaN(auchan)
minipreco = remove_NaN(minipreco)

### Explanation Comment

Now we have all the data in the final format, although we may have some differences across each data file since some products may not exist in all supermarkets that were chosen for this project

In [9]:
# First we will add the missing product rows to each data frame
#    To do this we will import the input data for the webscrapping scripts so that we can access all products that were search on
categories = pd.read_excel('Categories.xlsx')

# From this file we only need the 'Product' column buut to maintain on the DataFrame 
categories = pd.DataFrame(categories["Product"])

# And rename this column to match the names that we have on each supermarket data
categories.columns = ["Product Object"]


# To have all the necessary rows in every dataframe we need to to a left join using as the left data frame 
#    the categories file that was just imported
continente_AllProd = categories.merge(continente, on='Product Object', how='left')
auchan_AllProd = categories.merge(auchan, on='Product Object', how='left')
minipreco_AllProd = categories.merge(minipreco, on='Product Object', how='left')

In [10]:
# Since that depending the supermarket there may be new rows with NaN data, then is needed to re-apply the Remove_NaN function
#Apply the same data cleaning to the newly created dataframes that was applied previously

continente_AllProd = remove_NaN(continente_AllProd)
auchan_AllProd = remove_NaN(auchan_AllProd)
minipreco_AllProd = remove_NaN(minipreco_AllProd)

continente_AllProd["Supermarket"] = "Continente"
auchan_AllProd["Supermarket"] = "Auchan"
minipreco_AllProd["Supermarket"] = "Mini Preço"



In [11]:
# Now we are able to concatenate all the three data frames into one, preparing the final file

data_frames = [continente_AllProd, auchan_AllProd, minipreco_AllProd]

concat_data = pd.concat(data_frames)
concat_data.head()

,Product Object,Supermarket,Category Object,Average Per Product,Min Price per Product,Max Price per Product,Count of Products
0,Perú,Continente,Talho,7.337,3.99,8.99,10
1,Frango,Continente,Talho,5.25,2.39,8.99,10
2,Bacalhau,Continente,Peixaria,13.023,7.3,27.06,10
3,Cebola,Continente,Legumes,3.148,0.79,11.92,10
4,Batata,Continente,Legumes,1.518,0.56,3.29,10


In [12]:
## Some checks to see if the concatenation was correctly done
if(len(concat_data) == 50*3):
    print("Passed Check")
else:
    print("Review the data")

concat_data['Product Object'].value_counts().head()
#    should return only 3's since we have one entry for each of these products

Passed Check


Perú                3
Sacos do Lixo       3
Grão                3
Feijao              3
Detergente Loiça    3
Name: Product Object, dtype: int64

In [13]:
# Fill all the NaN that are on the Prices and Count Columns
concat_data.fillna(0, inplace = True)

In [14]:
# We want our data on a specific format so that the dashboard is able to read it
concat_data = concat_data.sort_values(by = 'Product Object')

In [15]:
# Export to Excel
concat_data.to_excel('Data_ForDashboard.xlsx', index=False)